# Reference

- [Transformer’s from scratch in simple python. Part-I](https://medium.com/@hhpatil001/transformers-from-scratch-in-simple-python-part-i-b290760c1040)

# 1. Prepare tokenizer


In [1]:
import torch
from transformers import AutoTokenizer


/Users/karlxu/Workspace/github/xuyuji9000/machine-learning-playground/04-ml-algorithms/06-transformer/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [23]:
text = 'I love data science.'
inputs = tokenizer(text, add_special_tokens=False, return_tensors='pt')

inputs['input_ids'].shape, inputs['input_ids']

(torch.Size([1, 5]), tensor([[1045, 2293, 2951, 2671, 1012]]))

# 2. Look into Bert model configuration

In [5]:
from transformers import AutoConfig

In [9]:
config = AutoConfig.from_pretrained('bert-base-uncased')

# config

# 3. Prepare the Embeddings

In [18]:
from torch import nn

token_embeddings= nn.Embedding(config.vocab_size, config.hidden_size)

print(token_embeddings.weight)

Parameter containing:
tensor([[-3.9642e-01, -2.5575e-01,  2.2973e+00,  ...,  1.5993e+00,
          3.3287e-01, -1.1616e+00],
        [ 1.1665e+00, -1.3761e+00,  3.6074e-02,  ...,  1.7865e-02,
         -5.6691e-01,  2.6453e+00],
        [-1.3022e+00,  6.4175e-02, -3.1888e-01,  ..., -6.9032e-01,
          1.0716e+00, -2.2049e+00],
        ...,
        [ 5.7730e-01, -1.3348e+00,  7.7632e-01,  ..., -6.1008e-01,
          1.4037e+00,  1.3111e+00],
        [ 1.5936e+00, -4.4314e-01, -4.2099e-01,  ...,  1.4291e+00,
         -2.9807e-01,  1.1399e+00],
        [ 1.3985e+00,  4.4623e-01, -2.1525e-03,  ..., -2.0247e-01,
          6.5869e-01,  9.2411e-01]], requires_grad=True)


In [21]:
inputs_embeds = token_embeddings(inputs.input_ids)

inputs_embeds.shape

torch.Size([1, 5, 768])

# 4. calculate attentions score

In [40]:
import torch
import torch.nn.functional as F
from math import sqrt


In [64]:
query  = key = value  = inputs_embeds


print(query.shape, key.transpose(1, 2).shape)

torch.bmm(query, key.transpose(1, 2)).shape

torch.Size([1, 5, 768]) torch.Size([1, 768, 5])


torch.Size([1, 5, 5])

In [ ]:
def scaled_dot_product_attention(query, key, value):
    dim_k = query.size(-1)

    # dot product: torch.Size([1, 5, 768]), torch.Size([1, 768, 5])
    # result: torch.Size([1, 5, 5])
    scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k) # torch.bmm is batch matrix - matrix multiplication. 
                                                                # Basically a dot product.
    
    # amplify the difference between highest and lowest
    weights = F.softmax(scores, dim=-1)

    # dot product: torch.Size([1, 5, 5]), torch.Size([1, 5, 768])
    # result: torch.Size([1, 5, 768])
    return torch.bmm(weights, value)


    